In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import nltk
import transformers
from nltk.corpus import stopwords
# nltk.download('stopwords')
sw_nltk = stopwords.words('english')
from gensim.models import Word2Vec, word2vec
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from PIL import Image
import natsort
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.colors as colors
from matplotlib.collections import PatchCollection

TF_ENABLE_ONEDNN_OPTS=0

2023-06-03 19:35:18.123094: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Load the data
train2 = pd.read_csv('./training1.csv', delimiter='\t')

IDs = []  #set1
images = []
directory = 'TRAINING'   # directory where we have images 
filenames = natsort.natsorted(os.listdir(directory))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        ID = filename
        pathname = os.path.join(directory, filename)
        im = Image.open(pathname)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
#             print("This is 1 channel, so we omit it", imnp.shape, filename)
            continue
        IDs.append(ID)
        images.append(imnp)

def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
list1 = IDs
list2 = list(train2.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(list1, list2)
print('len of common strings', len(common_strings))

sorted_ids = natsort.natsorted(common_strings)

print(sorted_ids)

# Sort the dataframe with natural ordering of the IDs
train2['prefix_file_name'] = train2['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
sorted_train_df = train2.sort_values(by='prefix_file_name', ascending=True)
sorted_train_df

# Assuming 'df' is your DataFrame and 'common_strings' is the list of strings
# Get the common string values in the column 
filtered_df = sorted_train_df[sorted_train_df['file_name'].isin(sorted_ids)].reset_index(drop=True)

# Print the filtered DataFrame
print('filtered_df shape', filtered_df.shape)

train3 = filtered_df.copy()
# print(train3.shape)

trainx = train3.rename(columns={'Text Transcription': 'text'})
display(trainx)



### LOAD TEST DATA

test1 = pd.read_csv('Test.csv', delimiter='\t')
test_labels = pd.read_csv('test_labels.txt', 
                          delimiter='\t',
                         header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

merged_test = pd.merge(test1, test_labels, on='file_name', how='inner')


# Sort the dataframe with natural ordering of the IDs
merged_test['prefix_file_name'] = merged_test['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
merged_test1 = merged_test.sort_values(by='prefix_file_name', ascending=True)
merged_test1


# # train = train.rename(columns={'Text Transcription': 'text'})
test2 = merged_test1.rename(columns={'Text Transcription': 'text'})
# test2



# ######################################################
train = trainx[['file_name', 'text', 'misogynous']]
train = train.rename(columns = {'misogynous':'label'})

test = test2[['file_name', 'text', 'misogynous']]
test = test.rename(columns = {'misogynous':'label'})

# train['label'] = train['label'].map({1: 'non_misogyn', 0: 'misogyn'})
# test['label'] = test['label'].map({1: 'non_misogyn', 0: 'misogyn'})

len of common strings 9986
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '30.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '39.jpg', '40.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '47.jpg', '48.jpg', '50.jpg', '52.jpg', '53.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '69.jpg', '70.jpg', '71.jpg', '73.jpg', '74.jpg', '75.jpg', '78.jpg', '79.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '87.jpg', '89.jpg', '90.jpg', '91.jpg', '92.jpg', '93.jpg', '94.jpg', '96.jpg', '97.jpg', '98.jpg', '99.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '108.jpg', '109.jpg', '111.jpg', '112.jpg', '113.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '118.

,file_name,misogynous,shaming,stereotype,objectification,violence,text,prefix_file_name
0,1.jpg,0,0,0,0,0,Milk Milk.zip,1
1,2.jpg,0,0,0,0,0,-What are you doing? -you told me to satanize ...,2
2,3.jpg,0,0,0,0,0,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,3
3,4.jpg,0,0,0,0,0,Bedroom Kitchen Bathroom Bron memes storage,4
4,5.jpg,0,0,0,0,0,WAKEUP EARLY FREELANCERS,5
...,...,...,...,...,...,...,...,...
9981,15002.jpg,0,0,0,0,0,WAITING FOR THE END OF THE COVID imgflip.com,15002
9982,15003.jpg,0,0,0,0,0,SMART WOMEN ARE AROUND imgflip.com,15003
9983,15004.jpg,0,0,0,0,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,15004
9984,15005.jpg,0,0,0,0,0,COOKING FOR MY WIFE imgflip.com,15005


In [3]:
train

,file_name,text,label
0,1.jpg,Milk Milk.zip,0
1,2.jpg,-What are you doing? -you told me to satanize ...,0
2,3.jpg,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,0
3,4.jpg,Bedroom Kitchen Bathroom Bron memes storage,0
4,5.jpg,WAKEUP EARLY FREELANCERS,0
...,...,...,...
9981,15002.jpg,WAITING FOR THE END OF THE COVID imgflip.com,0
9982,15003.jpg,SMART WOMEN ARE AROUND imgflip.com,0
9983,15004.jpg,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,0
9984,15005.jpg,COOKING FOR MY WIFE imgflip.com,0


In [4]:
test

,file_name,text,label
436,15001.jpg,G HIS. UNDYING FIDELITY Steve is hot and perfe...,0
854,15002.jpg,How limagined myself as a Teacher...... How I ...,0
743,15004.jpg,WHERE WILL YOU BE WHEN DIARRHEA STRIKE memecen...,0
44,15005.jpg,A MAN WITH DREAMS... NEEDS A WOMAN WITH VISION,0
873,15006.jpg,THIS IS HOW YOUR GIRLFRIEND SEES YOUR FEMALE F...,0
...,...,...,...
871,17078.jpg,There are multiple reasons to lower your car T...,0
683,17079.jpg,MICHELLE OBAMA IS A MAN IGUARANTEE IT makeamem...,1
406,17080.jpg,Looks like the airbags deployed 1234498,0
733,17081.jpg,Half woman half horse,0


In [5]:
import pickle
from sklearn.metrics import classification_report

# Define the path to the pickle file
train_pickle_path = "FUSION_Embeddings/fusion_embeddings_train.pkl"

# Load the embeddings from the pickle file
with open(train_pickle_path, 'rb') as f:
    fusion_embeddings_train = pickle.load(f)

    
# Define the path to the pickle file
test_pickle_path = "FUSION_Embeddings/fusion_embeddings_test.pkl"

# Load the embeddings from the pickle file
with open(test_pickle_path, 'rb') as f:
    fusion_embeddings_test = pickle.load(f)

print('fusion emb train shape', fusion_embeddings_train.shape)
print('fusion emb test shape', fusion_embeddings_test.shape)




# Prepare the labels
train['label']= train['label'].replace(['misogyn', 'non_misogyn'],[1,0])
# train_labels = train["label"].tolist()
test['label']= test['label'].replace(['misogyn', 'non_misogyn'],[1,0])
# test_labels = test["label"].tolist()

display(train)
display(test)


# # # Label Encoding
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

labelencoder.fit(train['label'])
train['encoded_label'] = labelencoder.fit_transform(train['label'])
test["encoded_label"] = labelencoder.transform(test['label'])

# # # Validate the mapping:
# train.groupby(["label", "encoded_label"]).aggregate("count")


fusion emb train shape (9986, 192)
fusion emb test shape (1000, 192)


,file_name,text,label
0,1.jpg,Milk Milk.zip,0
1,2.jpg,-What are you doing? -you told me to satanize ...,0
2,3.jpg,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,0
3,4.jpg,Bedroom Kitchen Bathroom Bron memes storage,0
4,5.jpg,WAKEUP EARLY FREELANCERS,0
...,...,...,...
9981,15002.jpg,WAITING FOR THE END OF THE COVID imgflip.com,0
9982,15003.jpg,SMART WOMEN ARE AROUND imgflip.com,0
9983,15004.jpg,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,0
9984,15005.jpg,COOKING FOR MY WIFE imgflip.com,0


,file_name,text,label
436,15001.jpg,G HIS. UNDYING FIDELITY Steve is hot and perfe...,0
854,15002.jpg,How limagined myself as a Teacher...... How I ...,0
743,15004.jpg,WHERE WILL YOU BE WHEN DIARRHEA STRIKE memecen...,0
44,15005.jpg,A MAN WITH DREAMS... NEEDS A WOMAN WITH VISION,0
873,15006.jpg,THIS IS HOW YOUR GIRLFRIEND SEES YOUR FEMALE F...,0
...,...,...,...
871,17078.jpg,There are multiple reasons to lower your car T...,0
683,17079.jpg,MICHELLE OBAMA IS A MAN IGUARANTEE IT makeamem...,1
406,17080.jpg,Looks like the airbags deployed 1234498,0
733,17081.jpg,Half woman half horse,0


In [6]:

import torch

train_labels = torch.tensor(train["encoded_label"].tolist())
test_labels = torch.tensor(test["encoded_label"].tolist())
train_labels


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


# # Encoding a sentence
# sent = "The quick brown fox jumped over the lazy dog."
# print(f"Tokenizer output a dictionary: {tokenizer(sent)}")

# # We can also decode ids to vocabulary
# print(tokenizer.decode([101, 1996, 4248, 2829, 4419, 5598, 2058, 1996, 13971, 3899, 1012, 102]))


train_encodings = tokenizer(
    train["text"].tolist(),
    padding=True,           # pad all inputs to max length
    max_length=80,         # Bert max is 512, we choose 24 for computational efficiency
    return_tensors="pt",    # Return format pytorch tensor
    truncation=True
)

print('train encodings shape ', train_encodings['input_ids'].shape)


test_encodings = tokenizer(
    test["text"].tolist(),
    padding=True,           # pad all inputs to max length
    max_length=80,         # Bert max is 512, we choose 24 for computational efficiency
    return_tensors="pt",    # Return format pytorch tensor
    truncation=True
)

print('test encodings shape ', test_encodings['input_ids'].shape)


from torch.utils.data import Dataset
import torch
from sklearn.preprocessing import LabelEncoder

# Define Custom Class for DistilBert Inputs
class RelationDataset(Dataset):
    
    def __init__(self, encodings: dict):  
        self.encodings = encodings
        
    def __len__(self) -> int:
        return len(self.encodings["input_ids"])
    
    def __getitem__(self, idx: int) -> dict:
        e = {k: v[idx] for k,v in self.encodings.items()}
        return e 

    
# Update encodings with labels
train_encodings["labels"] = train_labels
test_encodings["labels"] = test_labels

# Generate Datasets
train_ds = RelationDataset(train_encodings)
test_ds = RelationDataset(test_encodings)


from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Freeze embeddings
for name, param in model.distilbert.embeddings.named_parameters():
    param.requires_grad = False
    print(name, param.requires_grad)

# Freeze layers 1-4
freeze_layers = [1,2,3,4]
for name, param in model.distilbert.transformer.layer.named_parameters():
    if int(name[0]) in freeze_layers:    
        param.requires_grad = False
    print(name, param.requires_grad)

    
'''
Training
We're finally ready to train the model.
To train you need create training arguments object which contains the training details
like num_epochs, learning rate scheduling, batch size, etc.

The Trainer object handles the training loop setup and training.
'''

from transformers import TrainingArguments
from transformers import Trainer


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    save_strategy="epoch",
    lr_scheduler_type='cosine',
    per_device_train_batch_size = 32,
    fp16=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
)

trainer.train()




train encodings shape  torch.Size([9986, 80])
test encodings shape  torch.Size([1000, 80])


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

word_embeddings.weight False
position_embeddings.weight False
LayerNorm.weight False
LayerNorm.bias False
0.attention.q_lin.weight True
0.attention.q_lin.bias True
0.attention.k_lin.weight True
0.attention.k_lin.bias True
0.attention.v_lin.weight True
0.attention.v_lin.bias True
0.attention.out_lin.weight True
0.attention.out_lin.bias True
0.sa_layer_norm.weight True
0.sa_layer_norm.bias True
0.ffn.lin1.weight True
0.ffn.lin1.bias True
0.ffn.lin2.weight True
0.ffn.lin2.bias True
0.output_layer_norm.weight True
0.output_layer_norm.bias True
1.attention.q_lin.weight False
1.attention.q_lin.bias False
1.attention.k_lin.weight False
1.attention.k_lin.bias False
1.attention.v_lin.weight False
1.attention.v_lin.bias False
1.attention.out_lin.weight False
1.attention.out_lin.bias False
1.sa_layer_norm.weight False
1.sa_layer_norm.bias False
1.ffn.lin1.weight False
1.ffn.lin1.bias False
1.ffn.lin2.weight False
1.ffn.lin2.bias False
1.output_layer_norm.weight False
1.output_layer_norm.bias Fals

Using amp half precision backend
***** Running training *****
  Num examples = 9986
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1565


Step,Training Loss
500,0.468300
1000,0.315900
1500,0.235700


Saving model checkpoint to ./results/checkpoint-313
Configuration saved in ./results/checkpoint-313/config.json
Model weights saved in ./results/checkpoint-313/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-626
Configuration saved in ./results/checkpoint-626/config.json
Model weights saved in ./results/checkpoint-626/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-939
Configuration saved in ./results/checkpoint-939/config.json
Model weights saved in ./results/checkpoint-939/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1252
Configuration saved in ./results/checkpoint-1252/config.json
Model weights saved in ./results/checkpoint-1252/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1565
Configuration saved in ./results/checkpoint-1565/config.json
Model weights saved in ./results/checkpoint-1565/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1565, training_loss=0.3347306157072512, metrics={'train_runtime': 72.5022, 'train_samples_per_second': 688.669, 'train_steps_per_second': 21.586, 'total_flos': 1033452689827200.0, 'train_loss': 0.3347306157072512, 'epoch': 5.0})

In [7]:
import numpy as np
from sklearn.metrics import classification_report 

preds = trainer.predict(test_ds)
print(preds)


preds = labelencoder.inverse_transform(np.argmax(preds.predictions, axis=1))
print(classification_report(test["label"].tolist(), preds))

***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


PredictionOutput(predictions=array([[ 0.2308, -0.5386],
       [ 2.344 , -2.86  ],
       [-1.858 ,  1.668 ],
       ...,
       [-1.25  ,  1.172 ],
       [-0.9404,  0.82  ],
       [-1.687 ,  1.704 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 